# Simple Cone Search and Simple Image Access

__DESCRIPTION__

Script to use pyvo to perform a simple cone search and then retrieve a cutout of the object of interest using Simple Image Access Protocol.

@authors Javier Hernandez & Tamara Civera(CEFCA)

__Find out coordinate of object of interst__

In [8]:
import pyvo as vo
from astropy.coordinates import SkyCoord

pos = SkyCoord.from_name('NGC 7331')
print(pos)

<SkyCoord (ICRS): (ra, dec) in deg
    (339.2617, 34.43)>


__Execute a cone search__

In [10]:
scs_srv = vo.dal.SCSService("http://archive.cefca.es/catalogues/vo/cone/jplus-dr2/JPLUS.MAGABDUALOBJ")
scs_results = scs_srv.search(pos=pos, radius=10/3600)
print(scs_results)

<Table length=1>
TILE_ID-NUMBER TILE_ID NUMBER  ALPHA_J2000  DELTA_J2000   R_EFF    FWHM_WORLD  ...       MAG_ERR_APER_6_0         FLAGS        NORM_WMAP_VAL      MASK_FLAGS       MAG_PSFCOR             MAG_ERR_PSFCOR      
                                   deg          deg       pixel       deg      ...                                                                                    mag                                     
    object      int32  int32     float64      float64    float32    float32    ...         float32[12]          int32[12]       float32[12]       int32[12]       float32[12]              float32[12]        
-------------- ------- ------ ------------- ----------- --------- ------------ ... ---------------------------- --------- ----------------------- ---------- --------------------- ---------------------------
    89047-4991   89047   4991 137.424210133 37.60157391 31.244287 0.0035294592 ... 0.0043654097 .. 0.0065274057    0 .. 0 0.9616649 .. 0.97914416     0 .. 

In [11]:
print(scs_results.fieldnames)

('TILE_ID-NUMBER', 'TILE_ID', 'NUMBER', 'ALPHA_J2000', 'DELTA_J2000', 'R_EFF', 'FWHM_WORLD', 'A_WORLD', 'B_WORLD', 'THETA_J2000', 'ISOAREA_WORLD', 'RELERRA_WORLD', 'RELERRB_WORLD', 'ERRTHETA_J2000', 'CLASS_STAR', 'MAG_AUTO', 'MAG_ISO', 'KRON_RADIUS', 'MAG_APER_3_0', 'MAG_APER_6_0', 'MAG_ERR_AUTO', 'MAG_ERR_ISO', 'MAG_ERR_APER_3_0', 'MAG_ERR_APER_6_0', 'FLAGS', 'NORM_WMAP_VAL', 'MASK_FLAGS', 'MAG_PSFCOR', 'MAG_ERR_PSFCOR')


In [12]:
scs_results.fieldname_with_ucd('phys.angSize.smajAxis')

'A_WORLD'

__Compute object size__

In [13]:
obj = scs_results[0]
size = obj['A_WORLD'] * obj['KRON_RADIUS'] * 2 * 1.2
print(size)

0.026223427802324294


__Perform an image search__

In [ ]:
sia_srv = vo.dal.SIAService("http://archive.cefca.es/catalogues/vo/siap/jplus-dr2")
resultset = sia_srv.search(pos=pos, size=size)
resultset.to_table()

In [ ]:
print(resultset.fieldnames)

__Display RGB cutout__

In [ ]:
from IPython.display import Image, display

for row in resultset:
    if row['Format'] != 'image/png' or not 'cutout' in row['Name']:
        continue
    display(Image(url=row['Url']))
    break

__Send to ALadin/DS9 cutouts of all SDSS filter__

In [ ]:
for row in resultset:
    if row['Format'] != 'image/fits':
        continue
    if not 'cutout' in row['Name'] or 'WEIGHT' in row['Name']:
        continue
    if ('SDSS' in row['BandPass']):
        row.broadcast_samp()